In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
# load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
else:
    print("Api key found")


In [ ]:
openai = OpenAI()

In [ ]:
# To give you a preview -- calling OpenAI with these messages is this easy. 

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

In [ ]:
# A class to represent a Webpage
# some websites you need a proper header when fetching them
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the beutiful BeautifulSoup library 
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(['script', 'style', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        

In [ ]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

In [ ]:
# Define our system prompt
system_prompt = """
Your are an asistant that analyzes the content of a webpage and provides a short summary, ignoreing text that might be navigation related. Respond in markdown."""

In [ ]:
# a function writes a User Prompt that asks for summaries of website
def user_prompt_for(website):
    user_promt = f"You are looking at a website titled {website.title}"
    user_promt += "\nThe contents of this website is as follows;\
        please provide short summary of this website in markdown. \
            If it includes news or announcements, please include that as well."
    user_promt += website.text
    return user_promt

In [ ]:
print(user_prompt_for(ed))

In [ ]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2+2?"}
]

In [ ]:
# preview
response = openai.chat.completions.create(model='gpt-3.5-turbo', messages=messages)

In [ ]:
def message_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
message_for(ed)

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(model='gpt-3.5-turbo', messages=message_for(website))
    return response.choices[0].message.content


: 

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")